In [1]:
from os import listdir
from os.path import isfile, join

image_path = "./datasets/catsvsdogs/images/"

file_names = [f for f in listdir(image_path) if isfile(join(image_path, f))]


In [2]:
# no_of_images
print(len(file_names))

print(file_names[:5])

3002
['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg']


In [3]:
import cv2
import numpy as np
import sys
import os
import shutil

# split data into training and test

# save and read those data in a different directories  it reduce ram usage

dog_count = 0
cat_count = 0

training_size = 1000
test_size = 500

training_images = []
training_labels = []
test_images = []
test_labels = []

size = 150

dog_dir_train = './datasets/catsvsdogs/train/dogs/'
cat_dir_train = './datasets/catsvsdogs/train/cats/'
dog_dir_val = './datasets/catsvsdogs/validation/dogs/'
cat_dir_val = './datasets/catsvsdogs/validation/cats/'



def make_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

make_dir(dog_dir_train)
make_dir(cat_dir_train)
make_dir(dog_dir_val)
make_dir(cat_dir_val)





In [4]:
def getZeros(number):
    if number > 10 and number < 100:
        return "0"
    if number < 10:
        return "00"
    else:
        return ""

In [5]:
?print(file_names)

Object `print(file_names)` not found.


In [6]:
for i, file in enumerate(file_names):
#     print(i,file)
    if file_names[i][0] == 'd':
        dog_count +=1
        image = cv2.imread(image_path+file)
#         print(image_path+file)
#         print(image)
        image = cv2.resize(image,(size, size), interpolation = cv2.INTER_AREA)
        if dog_count <= training_size:
            training_images.append(image)
            training_labels.append(1)
#             zeros = dog_count > 10 and dog_count < 100 ? "0" :
            zeros = getZeros(dog_count)
            cv2.imwrite(dog_dir_train + "dog" + str(zeros) + str(dog_count) + ".jpg", image)
        
        if dog_count > training_size and dog_count <= training_size+test_size:
            test_images.append(image)
            test_labels.append(1)
            zeros = getZeros(dog_count)
            cv2.imwrite(dog_dir_val + "dog" + str(zeros) + str(dog_count-1000) + ".jpg", image)
       
    if file_names[i][0] == 'c':
        cat_count += 1
        image = cv2.imread(image_path+file)
        image = cv2.resize(image,(size, size), interpolation = cv2.INTER_AREA)
            
        if cat_count <= training_size:
            training_images.append(image)
            training_labels.append(0)
#             zeros = dog_count > 10 and dog_count < 100 ? "0" :
            zeros = getZeros(cat_count)
            cv2.imwrite(cat_dir_train + "cat" + str(zeros) + str(cat_count) + ".jpg", image)
            
        if cat_count > training_size and cat_count <= training_size+test_size:
            test_images.append(image)
            test_labels.append(0)
            zeros = getZeros(cat_count)
            cv2.imwrite(cat_dir_val + "cat" + str(zeros) + str(cat_count-1000) + ".jpg", image)
    
    if dog_count == training_size + test_size and cat_count == training_size + test_size:
        break;
        
            
            
            
            

In [7]:
# save images in NPZ file format

In [8]:
np.savez('cats_vs_dogs_traning_data.npz', np.array(training_images))
np.savez('cats_vs_dogs_traning_labels.npz', np.array(training_labels))
np.savez('cats_vs_dogs_test_data.npz', np.array(test_images))
np.savez('cats_vs_dogs_test_labels.npz', np.array(test_labels))

In [9]:
def load_data_training_and_test(data_set_name):
    npzfile = np.load(data_set_name + "_traning_data.npz")
    train = npzfile['arr_0']
    
    npzfile = np.load(data_set_name + "_traning_labels.npz")
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(data_set_name + "_test_data.npz")
    test = npzfile['arr_0']
    
    npzfile = np.load(data_set_name + "_test_labels.npz")
    test_labels = npzfile['arr_0']
    
    return (train, train_labels), (test, test_labels)
    

In [10]:
(x_train, y_train), (x_test, y_test) = load_data_training_and_test("cats_vs_dogs")

In [11]:
print(y_train.shape[0])


2000


In [12]:
(x_train, y_train), (x_test, y_test) = load_data_training_and_test("cats_vs_dogs")
  
#have to reshape label data from (2000,) to (2000, 1)
#have to reshape test data from (1000,) to (1000, 1)

y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


#normalize

x_train /= 255
x_test /= 255



print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
    
    
    
    
    
    

(2000, 150, 150, 3)
(2000, 1)
(1000, 150, 150, 3)
(1000, 1)


In [13]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [14]:
batch_size = 16
epochs = 25

img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(32,(3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
             optimizer = 'rmsprop',
             metrics = ['accuracy'])


print(model.summary())





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________

In [15]:
history = model.fit(x_train,y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                 validation_data = (x_test, y_test),
                 shuffle = True)




Train on 2000 samples, validate on 1000 samples
Epoch 1/25


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}}]]
	 [[metrics/accuracy/Identity/_93]]
0 successful operations.
0 derived errors ignored.

In [16]:
model.save('cats_vs_dog_v1.h5')

In [17]:
import cv2
import numpy as np
from keras.models import load_model

classifier = load_model('cats_vs_dog_v1.h5')

def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    if pred == "[0]":
        pred = "cat"
    if pred == "[1]":
        pred = "dog"
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (252, 70) , cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]

    imageL = cv2.resize(input_im, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
#     cv2.imshow("Test Image", imageL)

    input_im = input_im.reshape(1,150,150,3) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])

    draw_test("Prediction", res, imageL) 
    cv2.waitKey(0)

cv2.destroyAllWindows()